In [127]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import urllib.request

import pickle
from urllib.parse import unquote
import time

import re

import requests
import json

In [37]:

count = 0


user_ids = []
dates = []
text = []
scores = []


for i in range(33): # get 2 pages of reviews
    
    print(i)

    if i < 34:

        url = 'https://www.yelp.com/biz/amys-baking-company-scottsdale'

        if i > 0:
            url += '?start=' + str(20 * i)

        req = urllib.request.Request(url)
        html_content = urllib.request.urlopen(req).read()
        soup = BeautifulSoup(html_content, 'html.parser')

        reviews = soup.findAll('div', {'class': 'review review--with-sidebar'})
        user_ids += [i.attrs['data-signup-object'][8:] for  i in reviews]
        dates += [i.find('span', {'class': 'rating-qualifier'}).text.split('\n')[1] for i in reviews]
        text += [i.find('p').text for i in reviews]
        scores += [float(i.find('div', {'class': 'i-stars'}).attrs['title'][:3]) for i in reviews]



df = pd.DataFrame(columns = ['user_id', 'date', 'rating', 'text'])

df['user_id'] = user_ids
df['date'] = dates
df['rating'] = scores
df['text'] = text

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [39]:
pd.to_pickle(df, 'data/reviews.p')

In [50]:
unique_reviewer_ids = df.drop_duplicates()

In [53]:
user_reviews = pd.DataFrame(columns = ['user_id','name', 'url', 'price', 'categories', 'address', 'date', 'score','text'])

yelp_unique_reviewers = pd.DataFrame(columns = ['user_id', 'location', 'num_friends', 'num_reviews', 'num_photos'])

In [86]:
for n in range(424,len(unique_reviewer_ids)):
    
    print(n)
    
    max_num_pages = 20
    i = 0
    
    user_id = unique_reviewer_ids.loc[n, 'user_id']
    
    while i < max_num_pages:


        url = 'https://www.yelp.com/user_details_reviews_self?userid=' + user_id

        if i > 0:
            url += '&rec_pagestart=' + str(10 * i)
            
            
        req = urllib.request.Request(url)
        html_content = urllib.request.urlopen(req).read()
        soup = BeautifulSoup(html_content, 'html.parser')
            
            
        if i == 0:
            rev_count = soup.find('li', {'class': 'review-count'})
            number_of_reviews = int(re.search(r'\d+', rev_count.text).group())
            max_num_pages = np.minimum(10,int(number_of_reviews / 10))
            
            user_info = soup.find('div', {'class': 'user-profile_info'})
            try:
                location = user_info.find('h3', {'class': 'user-location'}).text
            except:
                location = ''
                
            num_friends = int(user_info.findAll('strong')[0].text)
            num_reviews = int(user_info.findAll('strong')[1].text)
            num_photos = int(user_info.findAll('strong')[2].text)
            
            yelp_unique_reviewers.loc[n] = [user_id, location, num_friends, num_reviews, num_photos]

        i += 1
        


        reviews = soup.findAll('div', {'class': 'review'})

        df = pd.DataFrame(columns = ['user_id','name', 'url', 'price', 'categories', 'address', 'date', 'score','text'])

        for u in range(len(reviews)):

            df.loc[u,'user_id'] = unique_reviewer_ids.loc[n, 'user_id']

            df.loc[u,'name'] = reviews[u].find('a', {'class': 'biz-name js-analytics-click'}).text

            df.loc[u,'url'] = reviews[u].find('a', {'class': 'biz-name js-analytics-click'})['href']

            try:
                df.loc[u,'price'] = len(reviews[u].find('span', {'class': 'business-attribute price-range'}).text)
            except:
                df.loc[u,'price'] = 0
            try:
                df.loc[u,'categories'] = reviews[u].find('span', {'class': 'category-str-list'}).text
            except:
                df.loc[u,'categories'] = []
            
            try:
                df.loc[u,'address'] = str(reviews[u].find('address'))
            except:
                df.loc[u,'address'] = ''
            
            try:
                df.loc[u,'date'] = re.search(r'\d+/\d+/\d+', reviews[u].find('span', {'class': 'rating-qualifier'}).text).group()
            except:
                df.loc[u,'date'] = ''
                
            try:
                df.loc[u,'score'] = float(reviews[u].find('div', {'class': 'i-stars'}).attrs['title'][:3])
            except:
                df.loc[u,'score'] = 0
            
            try:
                df.loc[u,'text'] = reviews[u].find('p', {'lang': "en"}).text
            except:
                df.loc[u,'text'] = ''
            
        user_reviews = pd.concat([user_reviews, df])   

        
        
#         time.sleep(3)
        
    if n%10 == 0:
        
        pd.to_pickle(user_reviews, 'data/user_reviews.p')


424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646


In [90]:
user_reviews.index = range(len(user_reviews))

In [91]:
pd.to_pickle(user_reviews, 'data/user_reviews.p')

In [96]:
unique_venues = user_reviews[['name', 'url', 'price', 'categories', 'address']].drop_duplicates('url')

In [100]:
def clean_address(u):
    return re.sub(' +',' ', re.sub('<.*>|\\n',' ', u))

unique_venues['clean_address'] = unique_venues['address'].apply(clean_address)

In [110]:
pd.to_pickle(user_reviews, 'data/unique_venues.p')

In [109]:
unique_venues.index = range(len(unique_venues))

In [111]:
bing_key = #BING_KEY

In [134]:
unique_venues['lat'] = 0
unique_venues['lon'] = 0

In [137]:
for n in range(10,len(unique_venues)):

    destination = unique_venues.loc[n,'clean_address'][1:-1]

    encodedDest = urllib.parse.quote(destination, safe='')
    url = 'http://dev.virtualearth.net/REST/v1/Locations/' + encodedDest + '?o=json&key=' + bing_key

    try:
        request = urllib.request.Request(url)
        response = urllib.request.urlopen(request)
        r = response.read().decode(encoding="utf-8-sig")
        result = json.loads(r)

        unique_venues.loc[n, ['lat', 'lon']] = result['resourceSets'][0]['resources'][0]['point']['coordinates']
    except:
        pass
    
    
    if n%100 == 0:
        print(n)
        pd.to_pickle(unique_venues, 'data/unique_venues.p')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600


In [138]:
pd.to_pickle(unique_venues, 'data/unique_venues.p')